In [2]:
import os
import openai
import streamlit as st
import pandas as pd
from llama_index import StorageContext, load_index_from_storage
from llama_index import (
    Document,
    GPTVectorStoreIndex,
    LLMPredictor,
    PromptHelper,
    ServiceContext,
    LangchainEmbedding
)

from langchain.chat_models import AzureChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from llama_index.prompts.prompts import QuestionAnswerPrompt

# APIキーなどの設定
openai.api_key = os.getenv("AZURE_OPENAI_KEY")
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT")
os.environ["OPENAI_API_KEY"] = os.getenv("AZURE_OPENAI_KEY")
os.environ["OPENAI_API_BASE"] = os.getenv("AZURE_OPENAI_ENDPOINT")
openai.api_type = 'azure'
openai.api_version = '2023-05-15'

In [3]:
#service_contextの生成
def create_service_context():
    # LLM Predictor
    llm_predictor = LLMPredictor(llm=AzureChatOpenAI(
        deployment_name='GPT35TURBO',         #デプロイ名
        max_tokens=3000,                        #最大トークン数
        temperature=1,                          #出力のランダム度合い
        openai_api_version=openai.api_version   #openaiのapiのバージョン情報
    ))

    # テキストの埋め込みベクトル変換(Embedding)に関する設定
    embeddings = LangchainEmbedding(OpenAIEmbeddings(
        engine="ADA",        #エンベディングに使うモデル
        chunk_size=1,                           #ここでのチャンクサイズはバッチサイズ
        openai_api_version=openai.api_version   #openaiのapiのバージョン情報
    ))

    # Prompt Helper（テキスト分割に関する設定）
    prompt_helper = PromptHelper(
        max_input_size=3000,    # 最大入力サイズ
        num_output=1000,        # LLMの出力サイズ
        chunk_size_limit=1000,  # 使用する最大チャンクサイズ（チャンク：テキストを細かく分割したもの）
        max_chunk_overlap=0,    # チャンクオーバーラップの最大トークン数
        separator="。"        # テキスト分割の区切り文字
    )

    # Service Contextの生成
    service_context = ServiceContext.from_defaults(
        llm_predictor=llm_predictor,    # LLM Predictor
        embed_model=embeddings,         # エンベディングについての設定
        prompt_helper=prompt_helper     # Prompt Helper
    )

    return service_context, prompt_helper

def llama_generate(index, query, top_k):
    """llama-indexによる回答の生成"""
    # 与えるコンテキスト（商品リストのうちクエリとの類似度が高いもの）をもとに回答をもとめるようなプロンプト
    QA_PROMPT_TMPL = (
        "私たちは以下の情報をコンテキスト情報として与えます。 \n"
        "---------------------\n"
        "{context_str}"
        "\n---------------------\n"
        "この情報をもとに質問に日本語で回答してください。: {query_str}\n"
    )
    qa_prompt = QuestionAnswerPrompt(QA_PROMPT_TMPL)
    
    # 回答生成
    
    # プロンプトと上位いくつまでの類似度を使用するか設定
    query_engine = index.as_query_engine(
        engine='GPT35TURBO',#←ここを変更
        text_qa_template=qa_prompt, # 上記のプロンプトを与える（デフォルトは英語文）
        similarity_top_k=top_k      # 参考情報（商品リスト）のうちクエリとの類似度上位何件を生成に利用するか
    )
    # 生成
    response = query_engine.query(query)

    return response

In [4]:
service_context, prompt_helper = create_service_context()
storage_context = StorageContext.from_defaults(persist_dir="./storage")


/opt/conda/lib/python3.11/site-packages/langchain/embeddings/openai.py:214: UserWarning: WARNING! engine is not default parameter.
                    engine was transferred to model_kwargs.
                    Please confirm that engine is what you intended.
  warnings.warn(


In [5]:
index = load_index_from_storage(storage_context, service_context=service_context)

In [6]:
index

In [7]:
# クエリ （description：アップロードした顧客情報)
query = f"あなたは顧客に商品を推薦する営業です。\
以下の顧客情報に一番適しているものを提案してください。\
その理由も回答してください。\
また、推薦する際に顧客が購入したくなるような文章を生成してください。\
顧客情報： \
出力形式は以下のようにしてください。\
適しているもの：\
選んだ理由：\
推薦メッセージ："

# llama-indexによる回答の生成
result = llama_generate(index=index, query=query, top_k=1)

In [8]:
result

Response(response='申し訳ございません。提供いただいた情報から顧客に適した商品を提案することはできません。情報が不足しているか、または提供された情報が理解不能である可能性があります。もし他に何か質問や情報があれば、お知らせください。それに基づいてお手伝いさせていただきます。', source_nodes=[NodeWithScore(node=Node(text=',   0.  +0.j        , # may vary0.  +0.j        ,   0.  +0.j        ],[-12.5+17.20477401j,   0.  +0.j        ,   0.  +0.j        ,0.  +0.j        ,   0.  +0.j        ],[-12.5 +4.0614962j ,   0.  +0.j        ,   0.  +0.j        ,0.  +0.j        ,   0.  +0.j        ],[-12.5 -4.0614962j ,   0.  +0.j        ,   0.  +0.j        ,0.  +0.j        ,   0.  +0.j        ],[-12.5-17.20477401j,   0.  +0.j        ,   0.  +0.j        ,0.  +0.j        ,   0.  +0.j        ]])', doc_id='93cb015a-0717-42cb-8871-38b0ee4dbab4', embedding=None, doc_hash='4b864b4818654d4111020e3913e605c37d84ec563a7fa573b709ac5d826baf17', extra_info=None, node_info={'start': 2667, 'end': 3130, '_node_type': '1'}, relationships={<DocumentRelationship.SOURCE: '1'>: '87035bfc-7f39-441b-afc4-58a631e1d235', <DocumentRelationship.PREVIOUS: '2